<a href="https://colab.research.google.com/github/NielsRogge/Transformers-Tutorials/blob/master/BERT/Fine_tuning_BERT_(and_friends)_for_multi_label_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning BERT (and friends) for multi-label text classification

In this notebook, we are going to fine-tune BERT to predict one or more labels for a given piece of text. Note that this notebook illustrates how to fine-tune a bert-base-uncased model, but you can also fine-tune a RoBERTa, DeBERTa, DistilBERT, CANINE, ... checkpoint in the same way. 

All of those work in the same way: they add a linear layer on top of the base model, which is used to produce a tensor of shape (batch_size, num_labels), indicating the unnormalized scores for a number of labels for every example in the batch.

## Load dataset

Next, let's download a multi-label text classification dataset from the [hub](https://huggingface.co/).

At the time of writing, I picked a random one as follows:   

* first, go to the "datasets" tab on huggingface.co
* next, select the "multi-label-classification" tag on the left as well as the the "1k<10k" tag (fo find a relatively small dataset).

Note that you can also easily load your local data (i.e. csv files, txt files, Parquet files, JSON, ...) as explained [here](https://huggingface.co/docs/datasets/loading.html#local-and-remote-files).



In [15]:
from datasets import load_dataset, DatasetDict
import os

data_dir = "./data/processed_data/scaffold/"
# Load the datasets
train_dataset = load_dataset('csv', data_files=os.path.join(data_dir, "train/metadata.csv"))['train']
valid_dataset = load_dataset('csv', data_files=os.path.join(data_dir, "validation/metadata.csv"))['train']
test_dataset = load_dataset('csv', data_files=os.path.join(data_dir, "test/metadata.csv"))['train']

# Create a DatasetDict
datasets = DatasetDict({
    'train': train_dataset,
    'valid': valid_dataset,
    'test': test_dataset
})

As we can see, the dataset contains 3 splits: one for training, one for validation and one for testing.

In [16]:
datasets

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'smiles', 'Hepatobiliary disorders', 'Metabolism and nutrition disorders', 'Product issues', 'Eye disorders', 'Investigations', 'Musculoskeletal and connective tissue disorders', 'Gastrointestinal disorders', 'Social circumstances', 'Immune system disorders', 'Reproductive system and breast disorders', 'Neoplasms benign, malignant and unspecified (incl cysts and polyps)', 'General disorders and administration site conditions', 'Endocrine disorders', 'Surgical and medical procedures', 'Vascular disorders', 'Blood and lymphatic system disorders', 'Skin and subcutaneous tissue disorders', 'Congenital, familial and genetic disorders', 'Infections and infestations', 'Respiratory, thoracic and mediastinal disorders', 'Psychiatric disorders', 'Renal and urinary disorders', 'Pregnancy, puerperium and perinatal conditions', 'Ear and labyrinth disorders', 'Cardiac disorders', 'Nervous system disorders', 'Injury, poisoning and pr

Let's check the first example of the training split:

In [18]:
example = datasets['train'][1]
example

{'Unnamed: 0': 1,
 'smiles': 'O=C1CN(N=CC2:C:C:C(C3:C:C:C([N+](=O)[O-]):C:C:3):O:2)C([O-])=N1',
 'Hepatobiliary disorders': 1,
 'Metabolism and nutrition disorders': 1,
 'Product issues': 0,
 'Eye disorders': 1,
 'Investigations': 1,
 'Musculoskeletal and connective tissue disorders': 1,
 'Gastrointestinal disorders': 1,
 'Social circumstances': 0,
 'Immune system disorders': 1,
 'Reproductive system and breast disorders': 0,
 'Neoplasms benign, malignant and unspecified (incl cysts and polyps)': 1,
 'General disorders and administration site conditions': 1,
 'Endocrine disorders': 0,
 'Surgical and medical procedures': 0,
 'Vascular disorders': 1,
 'Blood and lymphatic system disorders': 1,
 'Skin and subcutaneous tissue disorders': 1,
 'Congenital, familial and genetic disorders': 0,
 'Infections and infestations': 0,
 'Respiratory, thoracic and mediastinal disorders': 1,
 'Psychiatric disorders': 1,
 'Renal and urinary disorders': 1,
 'Pregnancy, puerperium and perinatal conditions'

The dataset consists of smiles, labeled with one or more ADRs. 

Let's create a list that contains the labels, as well as 2 dictionaries that map labels to integers and back.

In [24]:
labels = [label for label in datasets['train'].features.keys() if label not in ['Unnamed: 0', 'smiles']]
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
print(labels)
print(id2label)
print(label2id)

['Hepatobiliary disorders', 'Metabolism and nutrition disorders', 'Product issues', 'Eye disorders', 'Investigations', 'Musculoskeletal and connective tissue disorders', 'Gastrointestinal disorders', 'Social circumstances', 'Immune system disorders', 'Reproductive system and breast disorders', 'Neoplasms benign, malignant and unspecified (incl cysts and polyps)', 'General disorders and administration site conditions', 'Endocrine disorders', 'Surgical and medical procedures', 'Vascular disorders', 'Blood and lymphatic system disorders', 'Skin and subcutaneous tissue disorders', 'Congenital, familial and genetic disorders', 'Infections and infestations', 'Respiratory, thoracic and mediastinal disorders', 'Psychiatric disorders', 'Renal and urinary disorders', 'Pregnancy, puerperium and perinatal conditions', 'Ear and labyrinth disorders', 'Cardiac disorders', 'Nervous system disorders', 'Injury, poisoning and procedural complications']
{0: 'Hepatobiliary disorders', 1: 'Metabolism and nu

## Preprocess data

As models like BERT don't expect text as direct input, but rather `input_ids`, etc., we tokenize the text using the tokenizer. Here I'm using the `AutoTokenizer` API, which will automatically load the appropriate tokenizer based on the checkpoint on the hub.

What's a bit tricky is that we also need to provide labels to the model. For multi-label text classification, this is a matrix of shape (batch_size, num_labels). Also important: this should be a tensor of floats rather than integers, otherwise PyTorch' `BCEWithLogitsLoss` (which the model will use) will complain, as explained [here](https://discuss.pytorch.org/t/multi-label-binary-classification-result-type-float-cant-be-cast-to-the-desired-output-type-long/117915/3).

In [39]:
from transformers import AutoTokenizer
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def preprocess_data(examples):
  # take a batch of texts
  smiles = examples["smiles"]
  # encode them
  encoding = tokenizer(smiles, padding="max_length", truncation=True, max_length=512)
  # add labels
  labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(smiles), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]

  encoding["labels"] = labels_matrix.tolist()
  
  return encoding

In [40]:
encoded_dataset = datasets.map(preprocess_data, batched=True, remove_columns=datasets['train'].column_names)

Map:   0%|          | 0/1143 [00:00<?, ? examples/s]

Map:   0%|          | 0/142 [00:00<?, ? examples/s]

Map:   0%|          | 0/142 [00:00<?, ? examples/s]

In [46]:
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1143
    })
    valid: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 142
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 142
    })
})

In [57]:
example = encoded_dataset['train'][0]
print(example)
print(len(example['input_ids']))
print(example.keys())

{'input_ids': [101, 10507, 1006, 1027, 1051, 1007, 1050, 1006, 10507, 1006, 1051, 1007, 27166, 1006, 1039, 1006, 1039, 1007, 1027, 1051, 1007, 27723, 1024, 1039, 1006, 1045, 1007, 1024, 1039, 1006, 1039, 1006, 1027, 1051, 1007, 13316, 2278, 1006, 1051, 1007, 2522, 1007, 1024, 1039, 1006, 1045, 1007, 1024, 1039, 1006, 1039, 1006, 1027, 1051, 1007, 13316, 2278, 1006, 1051, 1007, 2522, 1007, 1024, 1039, 1024, 1015, 2072, 1007, 27723, 1024, 1039, 1006, 1045, 1007, 1024, 1039, 1006, 1039, 1006, 1027, 1051, 1007, 13316, 2278, 1006, 1051, 1007, 2522, 1007, 1024, 1039, 1006, 1045, 1007, 1024, 1039, 1006, 1039, 1006, 1027, 1051, 1007, 13316, 2278, 1006, 1051, 1007, 2522, 1007, 1024, 1039, 1024, 1015, 2072, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [58]:
tokenizer.decode(example['input_ids'])

'[CLS] cc ( = o ) n ( cc ( o ) cn ( c ( c ) = o ) c1 : c ( i ) : c ( c ( = o ) ncc ( o ) co ) : c ( i ) : c ( c ( = o ) ncc ( o ) co ) : c : 1i ) c1 : c ( i ) : c ( c ( = o ) ncc ( o ) co ) : c ( i ) : c ( c ( = o ) ncc ( o ) co ) : c : 1i [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD

In [29]:
example['labels']

[0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0]

In [59]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

['Metabolism and nutrition disorders',
 'Eye disorders',
 'Investigations',
 'Musculoskeletal and connective tissue disorders',
 'Gastrointestinal disorders',
 'Immune system disorders',
 'General disorders and administration site conditions',
 'Vascular disorders',
 'Blood and lymphatic system disorders',
 'Skin and subcutaneous tissue disorders',
 'Infections and infestations',
 'Respiratory, thoracic and mediastinal disorders',
 'Psychiatric disorders',
 'Renal and urinary disorders',
 'Ear and labyrinth disorders',
 'Cardiac disorders',
 'Nervous system disorders',
 'Injury, poisoning and procedural complications']

Finally, we set the format of our data to PyTorch tensors. This will turn the training, validation and test sets into standard PyTorch [datasets](https://pytorch.org/docs/stable/data.html). 

In [60]:
encoded_dataset.set_format("torch")

## Define model

Here we define a model that includes a pre-trained base (i.e. the weights from bert-base-uncased) are loaded, with a random initialized classification head (linear layer) on top. One should fine-tune this head, together with the pre-trained base on a labeled dataset.

This is also printed by the warning.

We set the `problem_type` to be "multi_label_classification", as this will make sure the appropriate loss function is used (namely [`BCEWithLogitsLoss`](https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html)). We also make sure the output layer has `len(labels)` output neurons, and we set the id2label and label2id mappings.

In [61]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", 
                                                           problem_type="multi_label_classification", 
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Train the model!

We are going to train the model using HuggingFace's Trainer API. This requires us to define 2 things: 

* `TrainingArguments`, which specify training hyperparameters. All options can be found in the [docs](https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments). Below, we for example specify that we want to evaluate after every epoch of training, we would like to save the model every epoch, we set the learning rate, the batch size to use for training/evaluation, how many epochs to train for, and so on.
* a `Trainer` object (docs can be found [here](https://huggingface.co/transformers/main_classes/trainer.html#id1)).

In [62]:
batch_size = 4
metric_name = "f1"

In [63]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    "BERT",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=4,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    #push_to_hub=True,
)

We are also going to compute metrics while training. For this, we need to define a `compute_metrics` function, that returns a dictionary with the desired metric values.

In [64]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
    
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

Let's verify a batch as well as a forward pass:

In [65]:
encoded_dataset['train'][0]['labels'].type()

'torch.FloatTensor'

In [66]:
encoded_dataset['train']['input_ids'][0]

tensor([  101, 10507,  1006,  1027,  1051,  1007,  1050,  1006, 10507,  1006,
         1051,  1007, 27166,  1006,  1039,  1006,  1039,  1007,  1027,  1051,
         1007, 27723,  1024,  1039,  1006,  1045,  1007,  1024,  1039,  1006,
         1039,  1006,  1027,  1051,  1007, 13316,  2278,  1006,  1051,  1007,
         2522,  1007,  1024,  1039,  1006,  1045,  1007,  1024,  1039,  1006,
         1039,  1006,  1027,  1051,  1007, 13316,  2278,  1006,  1051,  1007,
         2522,  1007,  1024,  1039,  1024,  1015,  2072,  1007, 27723,  1024,
         1039,  1006,  1045,  1007,  1024,  1039,  1006,  1039,  1006,  1027,
         1051,  1007, 13316,  2278,  1006,  1051,  1007,  2522,  1007,  1024,
         1039,  1006,  1045,  1007,  1024,  1039,  1006,  1039,  1006,  1027,
         1051,  1007, 13316,  2278,  1006,  1051,  1007,  2522,  1007,  1024,
         1039,  1024,  1015,  2072,   102,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0, 

In [67]:
#forward pass
outputs = model(input_ids=encoded_dataset['train']['input_ids'][0].unsqueeze(0), labels=encoded_dataset['train'][0]['labels'].unsqueeze(0))
outputs


SequenceClassifierOutput(loss=tensor(0.7169, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[ 0.0614, -0.0255,  0.3874, -0.2273,  0.1131, -0.1536, -0.5489,  0.1260,
          0.2135, -0.5155, -0.5011, -0.1642,  0.1320, -0.2568, -0.1461,  0.0250,
         -0.1430,  0.2871, -0.2254, -0.1758, -0.3216,  0.6229, -0.0373, -0.0956,
         -0.2543,  0.2034,  0.2254]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

Let's start training!

In [69]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["valid"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [70]:
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/ito/anaconda3/envs/project/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,No log,0.512176,0.814060,0.746308,0.000000
2,No log,0.499592,0.825867,0.748740,0.000000
3,No log,0.496753,0.827419,0.748022,0.014085
4,No log,0.494500,0.824654,0.748889,0.007042
5,No log,0.492412,0.831592,0.752020,0.028169


/home/ito/anaconda3/envs/project/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/ito/anaconda3/envs/project/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/ito/anaconda3/envs/project/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/ito/anaconda3/envs/project/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: W

TrainOutput(global_step=360, training_loss=0.5287789662679037, metrics={'train_runtime': 113.7452, 'train_samples_per_second': 50.244, 'train_steps_per_second': 3.165, 'total_flos': 1504017204710400.0, 'train_loss': 0.5287789662679037, 'epoch': 5.0})

## Evaluate

After training, we evaluate our model on the validation set.

In [71]:
trainer.evaluate()

/home/ito/anaconda3/envs/project/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 0.49241188168525696,
 'eval_f1': 0.8315916398713827,
 'eval_roc_auc': 0.7520197650622821,
 'eval_accuracy': 0.028169014084507043,
 'eval_runtime': 0.7373,
 'eval_samples_per_second': 192.591,
 'eval_steps_per_second': 12.206,
 'epoch': 5.0}

## Inference

Let's test the model on a new sentence:

In [47]:
text = "I'm happy I can finally train a model for multi-label classification"

encoding = tokenizer(text, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

outputs = trainer.model(**encoding)

The logits that come out of the model are of shape (batch_size, num_labels). As we are only forwarding a single sentence through the model, the `batch_size` equals 1. The logits is a tensor that contains the (unnormalized) scores for every individual label.

In [48]:
logits = outputs.logits
logits.shape

torch.Size([1, 11])

To turn them into actual predicted labels, we first apply a sigmoid function independently to every score, such that every score is turned into a number between 0 and 1, that can be interpreted as a "probability" for how certain the model is that a given class belongs to the input text.

Next, we use a threshold (typically, 0.5) to turn every probability into either a 1 (which means, we predict the label for the given example) or a 0 (which means, we don't predict the label for the given example).

In [57]:
# apply sigmoid + threshold
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())
predictions = np.zeros(probs.shape)
predictions[np.where(probs >= 0.5)] = 1
# turn predicted id's into actual label names
predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
print(predicted_labels)

['joy', 'optimism']
